# 1 Crear modelo Profile

- Para este caso nuestro modelo Profile va a tener una relacion uno a uno con user
- Se añaden los cam

In [ ]:
from django.contrib.auth.models import User
from django.db import models

class Profile(models.Model):
    user = models.OneToOneField(User, on_delete=models.CASCADE)
    phone_number = models.CharField(max_length=15, blank=True, null=True)
    address = models.TextField(blank=True, null=True)

    def __str__(self):
        return self.user.username


# 2 Creamos un ModelForm

forms.py

- Se utilizan UserRegistrationForm y ProfileForm en una sola clase
- phone_number y address capturan y se guardan dentro del mismo formulario usando el metodo save
- Se crea el Profile automaticamente despues de guardar el User

In [ ]:
from django import forms
from django.contrib.auth.models import User
from .models import Profile

class UserRegistrationForm(forms.ModelForm):
    password = forms.CharField(widget=forms.PasswordInput, label="Password")
    confirm_password = forms.CharField(widget=forms.PasswordInput, label="Confirm Password")

    phone_number = forms.CharField(max_length=15, label="Phone Number", required=False)
    address = forms.CharField(widget=forms.Textarea(attrs={"rows": 2}), label="Address", required=False)

    class Meta:
        model = User
        fields = ['username', 'email', 'password']

    def clean(self):
        cleaned_data = super().clean()
        password = cleaned_data.get("password")
        confirm_password = cleaned_data.get("confirm_password")

        if password != confirm_password:
            raise forms.ValidationError("Passwords do not match")
        return cleaned_data

    def save(self, commit=True):
        user = super().save(commit=False)
        user.set_password(self.cleaned_data['password'])  # Encripta la contraseña
        if commit:
            user.save()  # Guarda `User`
            Profile.objects.create(
                user=user,
                phone_number=self.cleaned_data.get('phone_number'),
                address=self.cleaned_data.get('address')
            )  # Crea `Profile` automáticamente
        return user


# Views.py

- Usamos solo un formulario para User y Profile

In [ ]:
from django.shortcuts import render, redirect
from .forms import UserRegistrationForm

def register(request):
    if request.method == "POST":
        form = UserRegistrationForm(request.POST)
        if form.is_valid():
            form.save()  # Guarda `User` y `Profile` con un solo método
            return redirect("forms/resgister")  # Redirigir al login después del registro
    else:
        form = UserRegistrationForm()

    return render(request, "register.html", {"form": form})


# Urls.py

In [ ]:
from django.urls import path
from .views import register

urlpatterns = [
    path("register/", register, name="register"),
]
